# Logistic Regression
---
This notebook uses Cirrus to run logistic regression on the Criteo dataset.

## Setup
---

In [1]:
# To ease development, each time a cell is run, all modules will be reloaded.
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys
import atexit

from cirrus import automate, lr

In [3]:
# Cirrus produces logs, but they will not show unless we add a handler that prints.
log = logging.getLogger("cirrus")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

## Instance, server, and task
---

First, we start an EC2 instance.

In [35]:
instance = automate.Instance(
    name="lr_example_instance",
    disk_size=32,
    typ="m5a.2xlarge",
    username="ubuntu",
    ami_name="cirrus_server_image",
    spot_bid="1.00"
)
instance.start()

__init__: Initializing EC2.
__init__: Resolving AMI name to AMI ID.
__init__: Done.
start: Checking if an instance with the same name is already running.
_exists: Listing instances.
_exists: An existing instance with the same name was found.
start: Done.


Second, we create a parameter server to run on our instance.

In [36]:
server = automate.ParameterServer(
    instance=instance,
    ps_port=1337,
    error_port=1338,
    num_workers=8
)

Third, we define our machine learning task.

In [37]:
task = lr.LogisticRegression(
    n_workers=16,
    n_ps=1,
    lambda_size=128,
    dataset="criteo-kaggle-19b",
    learning_rate=0.0001,
    epsilon=0.0001,
    progress_callback=None,
    train_set=(0, 6),
    test_set=(7, 8),
    minibatch_size=200,
    model_bits=19,
    ps=server,
    opt_method="adagrad",
    timeout=60
)

## Run
---

Next, we run our machine learning task.

In [38]:
task.run()

clear_lambda_logs: Listing log groups.
ClientManager: Initializing Cloudwatch Logs client.
clear_lambda_logs: Deleting matching log groups.
start: Uploading configuration.
run_command: Calling _connect_ssh.
_connect_ssh: Configuring.
_connect_ssh: Making connection attempt #1 out of 10.
run_command: Running `echo 'load_input_path: /mnt/efs/criteo_kaggle/train.csv 
load_input_type: csv
dataset_format: binary
num_classes: 2 
num_features: 13 
limit_cols: 14 
normalize: 0 
limit_samples: 10000 
s3_size: 50000 
use_bias: 1 
model_type: LogisticRegression 
minibatch_size: 200 
learning_rate: 0.000100 
epsilon: 0.000100 
model_bits: 19 
s3_bucket: criteo-kaggle-19b 
use_grad_threshold: 0 
grad_threshold: 0.001000 
train_set: 0-6 
test_set: 7-8' > config_1337.txt`.
run_command: Waiting for completion.
run_command: Fetching stdout and stderr.
run_command: stdout had length 0.
run_command: stderr had length 0.
run_command: Exit code was 0.
run_command: Done.
start: Starting parameter server.
ru

## Cleanup
---

When we're satisfied with the results, we kill our task.

In [39]:
task.kill()

run_command: Running `kill -n 9 $(cat error_1337.pid)`.
run_command: Waiting for completion.
run_command: Fetching stdout and stderr.
run_command: stdout had length 0.
run_command: stderr had length 93.
run_command: Exit code was 2.
run_command: Done.
run_command: Running `kill -n 9 $(cat ps_1337.pid)`.
run_command: Waiting for completion.
run_command: Fetching stdout and stderr.
run_command: stdout had length 0.
run_command: stderr had length 93.
run_command: Exit code was 2.
run_command: Done.


We also need to terminate our instance in order to avoid continuing charges.

In [40]:
instance.cleanup()

cleanup: Closing SSH client.
cleanup: Terminating instance.
cleanup: Waiting for instance to terminate.
cleanup: Deleting instance profile.
cleanup: Done.
launch_worker: Launching Task 1.
launch_worker: Launching Task 30001.
launch_worker: Launching Task 40001.
launch_worker: Launching Task 10001.
launch_worker: Launching Task 20001.
launch_worker: Launching Task 60001.
launch_worker: Launching Task 70001.
launch_worker: Launching Task 80001.
launch_worker: Launching Task 50001.
launch_worker: Launching Task 90001.
launch_worker: Launching Task 100001.
launch_worker: Launching Task 140001.
launch_worker: Launching Task 130001.
launch_worker: Launching Task 150001.
launch_worker: Launching Task 110001.
launch_worker: Launching Task 120001.


Exception in thread Worker 0:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/shea/Google Drive/-/Work/Cirrus/Repositories/Fixes2/python/frontend/cirrus/cirrus/automate.py", line 1495, in maintain_one
    launch_worker(lambda_name, task_id, config, n, ps)
  File "/Users/shea/Google Drive/-/Work/Cirrus/Repositories/Fixes2/python/frontend/cirrus/cirrus/automate.py", line 1435, in launch_worker
    "ps_ip": ps.public_ip(),
  File "/Users/shea/Google Drive/-/Work/Cirrus/Repositories/Fixes2/python/frontend/cirrus/cirrus/automate.py", line 805, in public_ip
    return self._instance.public_ip()
  File "/Users/shea/Google Drive/-/Work/Cirrus/Repositories/Fi

In [30]:
import os
os.system("aws iam remove-role-from-instance-profile --instance-profile-name lr_example_instance_instance_profile --role-name cirrus_instance_role")
os.system("aws iam delete-instance-profile --instance-profile-name lr_example_instance_instance_profile")

0